In [ ]:
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 30.8 MB/s eta 0:00:00


In [ ]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [ ]:
# ---------- ORACLES ----------
def oracle_constant(qc, ancilla, value=0):
    """Constant oracle: f(x)=0 or f(x)=1"""
    if value == 1:
        qc.x(ancilla)


def oracle_balanced_parity(qc, inputs, ancilla):
    """Balanced oracle: f(x) = x0 XOR x1 XOR ... XOR xn"""
    for q in inputs:
        qc.cx(q, ancilla)

In [ ]:
# ---------- DEUTSCH–JOZSA CIRCUIT ----------
def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """
    n: number of input qubits
    oracle_func: oracle function to modify the circuit
    oracle_args: extra arguments for oracle
    """
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize |0...0>|1>
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Oracle
    oracle_func(qc, *oracle_args)

    # Step 4: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 5: Measure only input qubits
    qc.measure(inputs, creg)

    return qc


In [ ]:
# ---------- EXECUTION ----------
def run_dj(qc):
    """Run Deutsch–Jozsa circuit on AerSimulator"""
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    if counts.get("0" * n, 0) == 1024:
        print("✅ Function is CONSTANT")
    else:
        print("✅ Function is BALANCED")


In [ ]:
# ---------- MAIN ----------
if __name__ == "__main__":
    n = 3  # number of input qubits

    print("\n=== Constant Oracle (f(x)=0) ===")
    qc_const = deutsch_jozsa_circuit(
        n, oracle_constant, n, 0
    )
    print(qc_const.draw(fold=-1))
    run_dj(qc_const)

    print("\n=== Balanced Oracle (Parity) ===")
    qc_balanced = deutsch_jozsa_circuit(
        n, oracle_balanced_parity, list(range(n)), n
    )
    print(qc_balanced.draw(fold=-1))
    run_dj(qc_balanced)



=== Constant Oracle (f(x)=0) ===
     ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ H ├┤M├──────
     ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├┤ H ├─╫─┤M├───
     ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├─╫──╫─┤M├
     ├───┤├───┤ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─╫──╫──╫─
     └───┘└───┘ ║  ║  ║ 
c: 3/═══════════╩══╩══╩═
                0  1  2 
Measurement counts: {'000': 1024}
✅ Function is CONSTANT

=== Balanced Oracle (Parity) ===
     ┌───┐          ┌───┐     ┌─┐           
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐   
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║       ║  ║ 
c: 3/══════════════════════════╩═══════╩══╩═
                               0       1  2 
Measurement counts: {'111': 1024}
✅ Function is BALANCED


###  Task 1 — Modify the Oracle

In [ ]:
def oracle_balanced_custom(qc, inputs, ancilla):
    """Custom balanced oracle: flips ancilla for half of all inputs."""
    n = len(inputs)
    # Example for n=3: Flip ancilla for inputs 001, 010, 100, 111
    # This can be achieved with CCX gates for specific inputs
    # and potentially CX gates for others.

    # Example for n=3: Flip for 001 (input 1)
    # Need to invert inputs 0 and 1, then apply CCX
    qc.x(inputs[0])
    qc.x(inputs[1])
    qc.ccx(inputs[0], inputs[1], ancilla)
    qc.x(inputs[0])
    qc.x(inputs[1]) # Revert inputs 0 and 1

    # Example for n=3: Flip for 010 (input 2)
    qc.x(inputs[0])
    qc.x(inputs[2])
    qc.ccx(inputs[0], inputs[2], ancilla)
    qc.x(inputs[0])
    qc.x(inputs[2]) # Revert inputs 0 and 2

    # Example for n=3: Flip for 100 (input 4)
    qc.x(inputs[1])
    qc.x(inputs[2])
    qc.ccx(inputs[1], inputs[2], ancilla)
    qc.x(inputs[1])
    qc.x(inputs[2]) # Revert inputs 1 and 2

    # Example for n=3: Flip for 111 (input 7)
    qc.ccx(inputs[0], inputs[1], ancilla)
    qc.cx(inputs[2], ancilla)
    qc.ccx(inputs[0], inputs[1], ancilla)

### Task 2 — Change the Number of Input Qubits

In [ ]:
# ---------- RUNNING WITH DIFFERENT N ----------
print("\n=== Running with different numbers of input qubits ===")

for n in [2, 4, 5]:
    print(f"\n--- Running with n = {n} ---")

    print("\nConstant Oracle:")
    qc_const_n = deutsch_jozsa_circuit(n, oracle_constant, n, 0)
    print(qc_const_n.draw(fold=-1))
    run_dj(qc_const_n)

    print("\nBalanced Oracle (Parity):")
    qc_balanced_n = deutsch_jozsa_circuit(n, oracle_balanced_parity, list(range(n)), n)
    print(qc_balanced_n.draw(fold=-1))
    run_dj(qc_balanced_n)


=== Running with different numbers of input qubits ===

--- Running with n = 2 ---

Constant Oracle:
     ┌───┐┌───┐┌─┐   
q_0: ┤ H ├┤ H ├┤M├───
     ├───┤├───┤└╥┘┌─┐
q_1: ┤ H ├┤ H ├─╫─┤M├
     ├───┤├───┤ ║ └╥┘
q_2: ┤ X ├┤ H ├─╫──╫─
     └───┘└───┘ ║  ║ 
c: 2/═══════════╩══╩═
                0  1 
Measurement counts: {'00': 1024}
✅ Function is CONSTANT

Balanced Oracle (Parity):
     ┌───┐          ┌───┐     ┌─┐   
q_0: ┤ H ├───────■──┤ H ├─────┤M├───
     ├───┤       │  └───┘┌───┐└╥┘┌─┐
q_1: ┤ H ├───────┼────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫─
     └───┘└───┘└───┘└───┘      ║  ║ 
c: 2/══════════════════════════╩══╩═
                               0  1 
Measurement counts: {'11': 1024}
✅ Function is BALANCED

--- Running with n = 4 ---

Constant Oracle:
     ┌───┐┌───┐┌─┐         
q_0: ┤ H ├┤ H ├┤M├─────────
     ├───┤├───┤└╥┘┌─┐      
q_1: ┤ H ├┤ H ├─╫─┤M├──────
     ├───┤├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├┤ H ├─╫──╫─┤M├───
     ├───┤├───

### Task 3 — Add Noise Simulation

In [ ]:
# ---------- EXECUTION (Corrected with Noise Simulation) ----------
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit_aer.noise import NoiseModel, depolarizing_error # Import necessary components

# Corrected run_dj function to accept noise_model
def run_dj_noisy(qc, noise_model=None):
    """Run Deutsch–Jozsa circuit on AerSimulator with optional noise"""
    simulator = AerSimulator()
    if noise_model:
        simulator = AerSimulator(noise_model=noise_model)

    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    # Note: With noise, the outcome might not be a single value with 1024 counts.
    # We can check the most frequent outcome or look for a dominant outcome.
    # For simplicity here, we'll still check for the ideal outcomes, but keep in mind
    # noise will cause deviations.
    if counts.get("0" * n, 0) > counts.get("1" * n, 0): # Simple check for constant-like behavior
         print("✅ Function is likely CONSTANT (dominant '0'*n outcome)")
    elif counts.get("1" * n, 0) > counts.get("0" * n, 0): # Simple check for balanced-like behavior
         print("✅ Function is likely BALANCED (dominant '1'*n outcome)")
    else:
         print("⚠️ Results are inconclusive (significant noise effect or different function type)")


# ---------- NOISE SIMULATION ----------
# Create a simple noise model
noise_model = NoiseModel()
# Add depolarizing error to all single qubit gates
error = depolarizing_error(0.05, 1) # 5% error rate
noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3', 'x', 'y', 'z', 'h', 's', 'sdg', 't', 'tdg'])
# Add depolarizing error to all two qubit gates
error_cx = depolarizing_error(0.1, 2) # 10% error rate
noise_model.add_all_qubit_quantum_error(error_cx, ['cx'])


print("\n=== Balanced Oracle with Noise Simulation ===")
# Using the previously defined balanced oracle circuit (qc_balanced from the main block)
# or you can create a new one with a different n
qc_balanced_noisy = deutsch_jozsa_circuit(3, oracle_balanced_parity, list(range(3)), 3)
print(qc_balanced_noisy.draw(fold=-1))
run_dj_noisy(qc_balanced_noisy, noise_model=noise_model)


=== Balanced Oracle with Noise Simulation ===
     ┌───┐          ┌───┐     ┌─┐           
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐   
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║       ║  ║ 
c: 3/══════════════════════════╩═══════╩══╩═
                               0       1  2 
Measurement counts: {'100': 11, '010': 13, '000': 31, '001': 44, '101': 45, '110': 42, '011': 105, '111': 733}
✅ Function is likely BALANCED (dominant '1'*n outcome)


###  Task 4 — Run on IBM Quantum Device


### Task 5 — Circuit Analysis

In [12]:
# Analyze the unitary of the balanced parity oracle
n = 3 # Using n=3 for demonstration
qreg = QuantumRegister(n + 1, "q")
qc_oracle = QuantumCircuit(qreg)

# Apply the balanced parity oracle to the circuit
oracle_balanced_parity(qc_oracle, list(range(n)), n)

# Print the unitary definition of the oracle
print("Unitary of the Balanced Parity Oracle:")
print(qc_oracle.to_gate().definition)

Unitary of the Balanced Parity Oracle:
                    
q_0: ──■────────────
       │            
q_1: ──┼────■───────
       │    │       
q_2: ──┼────┼────■──
     ┌─┴─┐┌─┴─┐┌─┴─┐
q_3: ┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘


The unitary of the balanced parity oracle for $n=3$ is shown above. This oracle implements the function $f(x) = x_0 \oplus x_1 \oplus x_2$, where $\oplus$ denotes the XOR operation.

The oracle works by applying a series of CNOT gates. Each CNOT gate flips the state of the ancilla qubit ($q_3$) if the corresponding input qubit ($q_0$, $q_1$, or $q_2$) is in the $|1\rangle$ state.

For example, if the input state is $|101\rangle$, the first CNOT with $q_0$ will flip the ancilla. The second CNOT with $q_1$ will not flip the ancilla (since $q_1$ is $|0\rangle$). The third CNOT with $q_2$ will flip the ancilla again. The net effect is that the ancilla is flipped twice, returning to its original state. The XOR sum of the input bits $1 \oplus 0 \oplus 1 = 0$, and the ancilla remains in its initial state (which was effectively $|-\rangle$ in the Deutsch-Jozsa algorithm setup).

If the input state is $|111\rangle$, all three CNOT gates will flip the ancilla. The ancilla will be flipped three times, resulting in a net flip. The XOR sum of the input bits $1 \oplus 1 \oplus 1 = 1$, and the ancilla is flipped.

In general, for any input state $|x\rangle = |x_0 x_1 \dots x_{n-1}\rangle$, the ancilla qubit will be flipped if and only if the XOR sum of the input bits is 1. This is precisely how the balanced parity oracle implements the function $f(x) = x_0 \oplus x_1 \oplus \dots \oplus x_{n-1}$. When the ancilla is in the $|-\rangle$ state, flipping it corresponds to multiplying the state by -1, which is the desired behavior for the oracle in the Deutsch-Jozsa algorithm.